# 1. Carga y exploración de datos (2 puntos)
# • Cargar los datos desde skincare_products.csv en un DataFrame de PySpark.
# • Mostrar las primeras filas del dataset.
# • Realizar un resumen estadístico de las variables numéricas

In [35]:
# Importar librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd

# Iniciar una sesión de Spark
spark = SparkSession.builder.appName("ClasificacionSkincare").getOrCreate()

# Cargar el dataset
df_skincare = spark.read.csv("skincare_products.csv", header=True, inferSchema=True)

# Mostrar las primeras filas del dataset
print("Primeras filas del DataFrame:")
df_skincare.show(5)
df_skincare.show()
# Realizar un resumen estadístico de las variables numéricas
print("Resumen estadístico de variables numéricas:")
df_skincare.describe().show()

Primeras filas del DataFrame:
+-----------------+-----------+---------+---+------------+
|     Ingredientes|Hidratación|Absorción|SPF|Tipo de Piel|
+-----------------+-----------+---------+---+------------+
|Ácido Hialurónico|       Alto|    Medio|  0|        Seco|
|          Retinol|       Bajo|     Alto|  0|       Graso|
|       Vitamina C|      Medio|    Medio| 30|       Mixto|
|        Aloe Vera|       Alto|     Bajo| 15|    Sensible|
|      Niacinamida|      Medio|    Medio|  0|       Mixto|
+-----------------+-----------+---------+---+------------+
only showing top 5 rows

+--------------------+-----------+---------+---+------------+
|        Ingredientes|Hidratación|Absorción|SPF|Tipo de Piel|
+--------------------+-----------+---------+---+------------+
|   Ácido Hialurónico|       Alto|    Medio|  0|        Seco|
|             Retinol|       Bajo|     Alto|  0|       Graso|
|          Vitamina C|      Medio|    Medio| 30|       Mixto|
|           Aloe Vera|       Alto|     Baj

# 2. Preprocesamiento de datos (2 puntos)
# • Convertir la columna "Tipo de Piel" en valores numéricos (0, 1, 2, 3).
# • Transformar las variables categóricas (Hidratación y Absorción) en valores numéricos.
# o Hidratación: Bajo (0), Medio (1), Alto (2)
# o Absorción: Bajo (0), Medio (1), Alto (2)
# • Unir todas las características en un vector usando VectorAssembler.

In [36]:
from pyspark.sql.functions import when, col

df_skincare=df_skincare.withColumn("label_Tipo_de_Piel", when (col("Tipo de Piel")=="Seco",0)
                                                        .when(col("Tipo de Piel")=="Graso",1)
                                                        .when(col("Tipo de Piel")=="Mixto",2)
                                                        .when(col("Tipo de Piel")=="Sensible",3))
df_skincare=df_skincare.withColumn("label_Hidratación", when (col("Hidratación")=="Bajo",0)
                                                        .when(col("Hidratación")=="Medio",1)
                                                        .when(col("Hidratación")=="Alto",2))
df_skincare=df_skincare.withColumn("label_Absorción",   when (col("Absorción")=="Bajo",0)
                                                        .when(col("Absorción")=="Medio",1)
                                                        .when(col("Absorción")=="Alto",2))
df_skincare.show()

from pyspark.ml.feature import VectorAssembler

feature_cols = ["label_Hidratación","label_Absorción","SPF"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(df_skincare)
data.show(5)

+--------------------+-----------+---------+---+------------+------------------+-----------------+---------------+
|        Ingredientes|Hidratación|Absorción|SPF|Tipo de Piel|label_Tipo_de_Piel|label_Hidratación|label_Absorción|
+--------------------+-----------+---------+---+------------+------------------+-----------------+---------------+
|   Ácido Hialurónico|       Alto|    Medio|  0|        Seco|                 0|                2|              1|
|             Retinol|       Bajo|     Alto|  0|       Graso|                 1|                0|              2|
|          Vitamina C|      Medio|    Medio| 30|       Mixto|                 2|                1|              1|
|           Aloe Vera|       Alto|     Bajo| 15|    Sensible|                 3|                2|              0|
|         Niacinamida|      Medio|    Medio|  0|       Mixto|                 2|                1|              1|
|           Ceramidas|       Alto|     Bajo|  0|        Seco|                 0|

# 3. División de datos y entrenamiento del modelo (3 puntos)
# • Dividir los datos en 80% entrenamiento y 20% prueba.
# • Entrenar un modelo de Árboles de Decisión con MLlib usando las características del dataset.


In [45]:
train, test = data.randomSplit([0.8, 0.2], seed=42)
print(f"Tamaño del conjunto de entrenamiento: {train.count()}")
print(f"Tamaño del conjunto de prueba: {test.count()}")

from pyspark.ml.classification import DecisionTreeClassifier

# Entrenar un modelo de Árboles de Decisión
dt = DecisionTreeClassifier(labelCol="label_Tipo_de_Piel", featuresCol="features")
model = dt.fit(train)
predictions = model.transform(test)

predictions.select("label_Tipo_de_Piel","prediction").show()

Tamaño del conjunto de entrenamiento: 15
Tamaño del conjunto de prueba: 5
+------------------+----------+
|label_Tipo_de_Piel|prediction|
+------------------+----------+
|                 2|       2.0|
|                 0|       0.0|
|                 2|       1.0|
|                 1|       1.0|
|                 1|       1.0|
+------------------+----------+



# 4. Predicción y evaluación (2 puntos)
# • Aplicar el modelo al conjunto de prueba y mostrar las predicciones.
# • Calcular la precisión del modelo usando MulticlassClassificationEvaluator.

In [44]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label_Tipo_de_Piel", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.60


# 5. Análisis de resultados y mejoras (1 punto)
# • Explicar en breve (3-5 líneas) qué tan preciso fue el modelo y cómo se podría mejorar (por ejemplo, usando otro algoritmo o ajustando parámetros).


# Análisis de Resultados

El modelo de Árbol de Decisión logró una precisión del 0.8 para un seed de 42, lo que indica que clasificó correctamente el 100% de los productos del conjunto de prueba. Este resultado puede deberse a la simplicidad del conjunto de datos o a la fuerte correlación entre las características y la etiqueta. Con un conjunto de datos más grande y complejo, es probable que la precisión sea más realista. Pero si se cambia el seed a 11, la clasificación bajo a un 71% y el programa

Posibles Mejoras

Ajuste de Parámetros: Se podrían ajustar los hiperparámetros del Árbol de Decisión, como la profundidad máxima (maxDepth) o el número mínimo de instancias por hoja (minInstancesPerNode), para evitar el sobreajuste.

Uso de Otros Algoritmos: Se podría probar con otros modelos de clasificación de MLlib, como Random Forest (un conjunto de árboles de decisión que a menudo mejora la precisión y reduce el sobreajuste) o Gradient-Boosted Trees, que tienden a ofrecer un mejor rendimiento.

Ingeniería de Características: Se podría analizar la columna de "Ingredientes" para extraer características adicionales que ayuden a la clasificación, por ejemplo, creando variables binarias para la presencia o ausencia de ciertos ingredientes clave.